# Regression

# Import required libraries

In [13]:
#!pip install xgboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import math
#from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import  tree, linear_model
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plot
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
import joblib
import pickle
import pyodbc
import pandas.io.sql as psql
from sklearn.model_selection import KFold

# Step1: Combining all the excel sheets

In [2]:
df=pd.read_csv('C:/Users/v-savrav/Desktop/30/InvociePrediction30days_02Oct2019_v1.csv')

In [3]:
df.columns

Index(['Country', 'Region', 'Frequency', 'CC', 'received_days',
       'processing_days', 'days_to_approval', 'days_to_be_cleared', 'PaidDays',
       'ReceiptDate', 'FirstPOCAssignmentDate', 'LastBusinessUserDate',
       'NewPaymentTerm', 'DocumentNo', 'Type', 'Doc. Date',
       'Amount in doc. curr.', 'Curr.', 'Clrng doc.', 'Pmnt date', 'Reference',
       'Clearing', 'Vendor', 'Pstng Date', 'CoCd', 'Net due dt', 'Year',
       'Year/month', 'Period', 'Discount base amount', 'Discount amount',
       'Bline Date', 'Reference Key', 'Entry Date', 'SubmitDate'],
      dtype='object')

In [4]:
df['NewPaymentTerm'].value_counts()

30.0    224590
Name: NewPaymentTerm, dtype: int64

# Step2: Dropping unused columns

In [5]:
#merge_df = merge_df.dropna(subset=['days_to_approval'],axis=0) #how='any',merge_df = merge_df.dropna(subset=['received_days'],axis=0) #how='any',
df.drop(['ReceiptDate', 'FirstPOCAssignmentDate', 'LastBusinessUserDate',
        'Doc. Date',
       'Clrng doc.', 'Pmnt date', 'Reference',
       'Clearing', 'Pstng Date', 'CoCd', 'Net due dt', 'Year',
       'Year/month', 'Period', 
       'Bline Date', 'Reference Key', 'Entry Date', 'SubmitDate'
], axis=1, inplace=True)

In [16]:
#Input Columns taking from Data , 
X=df[['CC','Vendor',
       'NewPaymentTerm', 'Amount in doc. curr.','Curr.','Type',  
            'Country',
       'Region', 'Frequency']]

In [17]:
X['Type']=pd.get_dummies(X['Type'])

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
X['NewPaymentTerm'].value_counts()

30.0    224590
Name: NewPaymentTerm, dtype: int64

# Step3: Preprocessing

In [19]:
from sklearn import  preprocessing # used for label encoding and imputing NaNs

# One-hot encode the data using pandas get_dummies
X.fillna(0,inplace=True)
for c in X.columns:
    if X[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X[c].values)) 
        X[c] = lbl.transform(list(X[c].values))

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Train Test Split

In [20]:
X_train, X_test = train_test_split(X,test_size = 0.25, random_state = 42)

In [14]:
dtrain = xgboost.DMatrix(X_train, label=y_train)
dtest = xgboost.DMatrix(X_test, label=y_test)

NameError: name 'y_train' is not defined

In [15]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

NameError: name 'params' is not defined

In [23]:
len(X_test)

56245

In [21]:
# Split the data into training and testing sets
def model_prediction(X_train,X_test,target,model_name):
    
    y_train, y_test = train_test_split(target, test_size = 0.25, random_state = 42)
    xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                               colsample_bytree=1, max_depth=7)
    params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 4,
          'learning_rate': 0.05, 'loss': 'ls'}
    dtrain = xgboost.DMatrix(X_train, label=y_train)
    dtest = xgboost.DMatrix(X_test, label=y_test)
    clf = ensemble.GradientBoostingRegressor(**params)
    xgb.fit(X_train,y_train)
    clf.fit(X_train,y_train)
    filename = 'C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model.sav'
    #joblib.dump(clb, filename)
    pickle.dump(clf, open(filename, 'wb'))
    filename1 = 'C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model1.sav'
    #joblib.dump(xgb, filename)
    pickle.dump(xgb, open(filename1, 'wb'))
    pred1 = xgb.predict(X_test)
    pred2 = clf.predict(X_test)
    finalpred =(pred1+pred2)/2
    mae = mean_absolute_error(finalpred ,y_test)
    return finalpred, y_test,mae


# Recieve

In [37]:
df.to_excel('C:/Users/v-savrav/Desktop/30/Data_test/target.xlsx')

In [25]:
len(df['received_days'])

224979

In [23]:
target = df['received_days']
receive_days_prediction ,actual_receive,mae1= model_prediction(X_train,X_test,target,'Stage1receive')

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:28:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
mae1

5.919190707631437

In [30]:
Receivedf=pd.DataFrame({'Rcvd -Act':actual_receive, 'Rcvd  -Pr':receive_days_prediction})


In [31]:
Receivedf

,Rcvd -Act,Rcvd -Pr
23216,3,6.125608
207519,15,23.724851
219777,37,13.060012
213486,4,5.724839
89316,0,3.801804
207085,-1,2.315560
134447,0,4.613128
75778,35,14.853744
83353,40,21.841778
19968,0,1.445805


# Process

In [32]:
target = df['processing_days']
processing_days_prediction,actual_process,mae2= model_prediction(X_train,X_test,target,'stage1_process')

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:54:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [33]:
mae2

4.962478087853626

In [34]:
processdf=pd.DataFrame({'Procesing - Act':actual_process, 'Procesing  - Pr':processing_days_prediction})

# Approve

In [35]:
target = df['days_to_approval']
approval_days_prediction,actual_approve,mae3= model_prediction(X_train,X_test,target,'stage1_approve')


C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:02:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [36]:
mae3

4.910875628931028

In [43]:
approvedf=pd.DataFrame({'Approval - Act':actual_approve, 'Approval - Pr':approval_days_prediction})

# Paiddays

In [38]:
target=df['PaidDays']
paid_days_prediction,actual_paid,mae4= model_prediction(X_train,X_test,target,'stage1_paid')

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:11:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [44]:
mae4

2.6988950500304916

In [51]:
paiddf = pd.DataFrame({'Paid - Act':actual_paid, 'Paid - Pr':paid_days_prediction})

In [48]:
len(X_test)

56245

In [42]:
X_test.columns

Index(['CC', 'Vendor', 'NewPaymentTerm', 'Amount in doc. curr.', 'Curr.',
       'Type', 'Discount amount', 'DocumentNo', 'Discount base amount',
       'Country', 'Region', 'Frequency'],
      dtype='object')

In [ ]:
X_test.to_excel('C:/Users/v-savrav/Desktop/30/Data_test/Stage1.xlsx')

In [ ]:
C:\\Users\\v-savrav\\Desktop\30\Train_results

In [52]:
frames=[Receivedf,processdf,approvedf,paiddf]
result = pd.concat(frames,axis=1)
final=pd.concat([result, X_test],axis=1)


In [53]:
final

,Rcvd -Act,Rcvd -Pr,Procesing - Act,Procesing - Pr,Approval - Act,Approval - Pr,Paid - Act,Paid - Pr,CC,Vendor,NewPaymentTerm,Amount in doc. curr.,Curr.,Type,Discount amount,DocumentNo,Discount base amount,Country,Region,Frequency
23216,3,6.125608,4,4.508636,4,4.110589,28,27.097280,1075,3009611,30.0,-1.289880e+03,8,0,0.00,3700002779,-1.209980e+03,12,6,1
207519,15,23.724851,0,3.832785,0,4.057480,26,24.285010,1962,2252281,30.0,-8.045352e+04,25,1,0.00,5100003468,-8.045352e+04,37,0,1
219777,37,13.060012,7,3.291935,7,3.235469,29,28.446464,1290,3009165,30.0,-3.745000e+01,51,0,0.00,3700024336,-3.500000e+01,72,0,1
213486,4,5.724839,30,11.930483,30,12.521400,33,27.625186,1705,3013865,30.0,-3.446165e+05,18,0,0.00,5100018759,-3.446165e+05,56,2,1
89316,0,3.801804,4,6.980041,4,6.671359,29,26.371493,1033,2328646,30.0,-2.736402e+06,11,0,0.00,5100111106,-2.820300e+06,15,3,1
207085,-1,2.315560,13,8.684268,13,8.349064,28,28.587046,1290,2284202,30.0,-1.305650e+03,54,0,0.00,5100293883,-1.305650e+03,72,0,1
134447,0,4.613128,11,6.448520,11,6.150522,22,21.402507,1035,2253661,30.0,-6.415200e+04,59,0,0.00,5100154850,-6.415200e+04,76,4,12
75778,35,14.853744,2,7.969849,2,7.520384,29,27.226091,1098,2248347,30.0,-7.975250e+04,25,1,0.00,5100240444,-7.975250e+04,37,0,1
83353,40,21.841778,7,3.475387,7,3.404043,26,27.606776,1291,3009165,30.0,-1.551000e+02,51,0,0.00,3700006056,-1.449800e+02,72,0,1
19968,0,1.445805,3,7.575376,3,8.165451,26,27.917660,1088,2108583,30.0,-3.132000e+04,37,0,0.00,5100153074,-2.700000e+04,52,3,1


In [56]:
final.to_excel('C:\\Users\\v-savrav\\Desktop\\30\\Train_results\\stage1_results.xlsx')

In [ ]:
ff=final.rename(columns={'Amount in doc. curr.':'Amount',
             'NewPaymentTerm':'Pay Term'})
                     

In [ ]:
final = ff[['DocumentNo','Rcvd -Act','Rcvd  -Pr','Procesing - Act','Procesing  - Pr',
'Approval - Act','Approval - Pr','Paid - Act','Paid - Pr','CC','Vendor',
'Pay Term','Amount','Curr.','Type','Discount amount','Country','Region','Frequency']]

In [ ]:
final['stage']='Stage1'

In [ ]:
final=final.dropna()

In [ ]:
stage1=final

In [ ]:
stage1

# Writing back to SQL Server

In [ ]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=azgovprdsql11;'
                      'Database=PWT;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
for index,row in stage1.iterrows():
    cursor.execute('''INSERT INTO [All_Payment_Term_Train_Stage1_Output]
                   (
                  [DocumentNo],
                  [Rcvd -Act],[Rcvd  -Pr],
                  [Procesing - Act],[Procesing  - Pr],
                [Approval - Act],[Approval - Pr],
                [Paid - Act],[Paid - Pr],
                [CC],[Vendor],[Pay Term],[Amount],[Curr.],[Type],
                [Discount amount],[Country],[Region],[Frequency],[stage] )
                   
    values (?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?)''',
    row['DocumentNo'],row['Rcvd -Act'],row['Rcvd  -Pr'],
                      row['Procesing - Act'],row['Procesing  - Pr'],
                      row['Approval - Act'],row['Approval - Pr'],
                      row['Paid - Act'],row['Paid - Pr'],
                      row['CC'],row['Vendor'],row['Pay Term'],row['Amount'],row['Curr.'],
                      row['Type'],row['Discount amount'],row['Country'],row['Region'],
                      row['Frequency'],row['stage'] )

In [ ]:
conn.commit()
conn.close()